In [46]:
import librosa
import numpy as np
import os

In [47]:
# ฟังก์ชันสกัดคุณลักษณะจากไฟล์เสียง
def extract_features(file_name, sr=44100, duration=30):
    y, sr = librosa.load(file_name, sr=sr)  # โหลดไฟล์เสียง
    num_segments = len(y) // (sr * duration)  # คำนวณจำนวน segment ในไฟล์เสียง
    features_list = []
    
    # สกัดคุณลักษณะจากแต่ละ segment
    for i in range(num_segments):
        start = i * sr * duration
        end = start + sr * duration
        y_segment = y[start:end]
        
        features = {}
        features['mfcc'] = np.mean(librosa.feature.mfcc(y=y_segment, sr=sr, n_mfcc=13).T, axis=0).tolist()
        features['spectral_centroid'] = np.mean(librosa.feature.spectral_centroid(y=y_segment, sr=sr).T, axis=0).tolist()
        features['zero_crossing_rate'] = np.mean(librosa.feature.zero_crossing_rate(y_segment).T, axis=0).tolist()
        features['spectral_bandwidth'] = np.mean(librosa.feature.spectral_bandwidth(y=y_segment, sr=sr).T, axis=0).tolist()
        features['chroma_stft'] = np.mean(librosa.feature.chroma_stft(y=y_segment, sr=sr).T, axis=0).tolist()
        mel_spectrogram = librosa.feature.melspectrogram(y=y_segment, sr=sr)
        features['mel_spectrogram'] = np.mean(librosa.power_to_db(mel_spectrogram).T, axis=0).tolist()
        
        features_list.append(features)
    
    return features_list

In [48]:
from tensorflow.keras.models import load_model
import pickle

# โหลดโมเดล
model = load_model('../you_path/Music_6_Genre_Full_Song_CNN_Model.h5')

# โหลด Label Encoder
with open('../you_path/Label_Encoder_6_Genre_Full_Song.pkl', 'rb') as file:
    label_encoder = pickle.load(file)

# โหลด Scaler
with open('../you_path/Scaler_6_Genre_Full_Song.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [49]:
# ฟังก์ชันเตรียมคุณลักษณะสำหรับการพยากรณ์
def prepare_features(features_list):
    X = []
    for features in features_list:
        feature_array = []
        for feature in features.values():
            feature_array.extend(feature)
        X.append(feature_array)
    
    X = np.array(X)  # แปลงเป็น numpy array
    X = scaler.transform(X)  # ทำการ Normalize ข้อมูล
    X = X.reshape(X.shape[0], X.shape[1], 1)  # ปรับ reshape ข้อมูลเพื่อให้ตรงกับรูปแบบของโมเดล
    return X

In [50]:
# ฟังก์ชันพยากรณ์แนวเพลงจากไฟล์เสียง
def predict_genre(file_name):
    features_list = extract_features(file_name)  # สกัดคุณลักษณะจากไฟล์เสียง
    X = prepare_features(features_list)  # เตรียมคุณลักษณะสำหรับการพยากรณ์
    predictions = model.predict(X)  # พยากรณ์แนวเพลง
    predicted_classes = np.argmax(predictions, axis=1)  # หาคลาสที่มีความน่าจะเป็นสูงสุด
    predicted_genres = label_encoder.inverse_transform(predicted_classes)  # แปลงค่าที่พยากรณ์กลับเป็นชื่อแนวเพลง
    
    for i, (pred, pred_class) in enumerate(zip(predictions, predicted_classes)): #zip เป็นฟังก์ชันที่นำข้อมูลจากสองชุดมาจับคู่กันเป็นคู่ๆ
        genre_probabilities = {label_encoder.classes_[j]: prob for j, prob in enumerate(pred)}
        genre_probabilities = dict(sorted(genre_probabilities.items(), key=lambda item: item[1], reverse=True))
        
        # แปลงค่าความคล้ายคลึงเป็นเปอร์เซ็นต์
        genre_percentages = {genre: f"{prob*100:.2f}%" for genre, prob in genre_probabilities.items()}
        
        print(f"Segment {i+1}: {predicted_genres[i]}")
        print(f"Probabilities: {genre_percentages}")
        
        if max(pred) < 0.5:  # ถ้าเปอร์เซ็นต์ความคล้ายคลึงสูงสุดน้อยกว่า 50%
            print("เป็นเพลงประเภทอื่นๆ")
        print("")

# ทำนาย
file_name = './you_path'
predict_genre(file_name)

1/1 [==============================] - 0s 67ms/step
Segment 1: Jazz
Probabilities: {'Jazz': '100.00%', 'HipHop': '0.00%', 'Metal': '0.00%', 'Classical': '0.00%', 'Rock': '0.00%', 'Country': '0.00%'}

Segment 2: Metal
Probabilities: {'Metal': '83.43%', 'Rock': '16.57%', 'Classical': '0.00%', 'Jazz': '0.00%', 'HipHop': '0.00%', 'Country': '0.00%'}

Segment 3: Rock
Probabilities: {'Rock': '100.00%', 'Metal': '0.00%', 'Jazz': '0.00%', 'HipHop': '0.00%', 'Classical': '0.00%', 'Country': '0.00%'}

Segment 4: Rock
Probabilities: {'Rock': '100.00%', 'Metal': '0.00%', 'Jazz': '0.00%', 'Classical': '0.00%', 'HipHop': '0.00%', 'Country': '0.00%'}

Segment 5: Rock
Probabilities: {'Rock': '100.00%', 'Metal': '0.00%', 'Jazz': '0.00%', 'Classical': '0.00%', 'HipHop': '0.00%', 'Country': '0.00%'}

Segment 6: Metal
Probabilities: {'Metal': '100.00%', 'Rock': '0.00%', 'Jazz': '0.00%', 'Classical': '0.00%', 'HipHop': '0.00%', 'Country': '0.00%'}

Segment 7: Rock
Probabilities: {'Rock': '100.00%', 'Metal'